In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torch.optim.lr_scheduler import CosineAnnealingLR, OneCycleLR


import numpy as np
import matplotlib.pyplot as plt



In [ ]:
BATCH_SIZE = 32

In [ ]:
transform = transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])
class MNISTWithAugmentation(torch.utils.data.Dataset):
    def __init__(self,
                 root = './data',
                 train = True, 
                 transform = None,
                 download = True):
        
        self.mnist = MNIST(root=root,
                           train=train,
                           transform=None,
                           download=download)
        
        self.base_transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize(mean=(0.5,), std=(0.5,))
                            ])
        self.transform = transform

    def __len__(self):
        return len(self.mnist)

    def __getitem__(self, index):
        image, label = self.mnist[index]
        image = self.base_transform(image)
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# Image processing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

# # MNIST dataset
# train_dataset = MNIST(root='./data', 
#                       train=True, 
#                       transform=transform,
#                       download=True)

train_dataset = MNISTWithAugmentation(root = './data', 
                                  train = True, 
                                  transform = transforms.Compose([
        transforms.RandomAffine(degrees = 20, translate = (0.1,0.1), scale = (0.9, 1.1))
    ]),
                                  download = True)


test_dataset = MNIST(root='./data', 
                     train=False, 
                     transform=transform,
                     download=True)


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

In [ ]:
def denorm(x):
    out = (x + 1) / 2
    # print(out)
    return out.clamp(0, 1)

In [ ]:
inputs, classes = next(iter(train_loader))
inputs = inputs[:6]
inputs = [el[0] for el in inputs]
classes = classes[:6]

fig = plt.figure()
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(denorm(inputs[i]), cmap='gray', interpolation='none');
    plt.title("Ground Truth: {}".format(classes[i]))
    # plt.xticks([])
    # plt.yticks([])
    
plt.tight_layout()
plt.show()

In [ ]:
IMAGE_SIZE = 784
# HIDDEN_SIZE = 256
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
from modules.mnist_classifier import train
from modules.mnist_models import CNN, CNN2, CNN3

In [ ]:
import math
NUM_BATCHES = math.ceil(len(train_loader.dataset)/train_loader.batch_size)

In [ ]:
# NUM_BATCHES

In [ ]:
classifier = CNN3().to(device=device)
loss_function = nn.CrossEntropyLoss()

learning_rate = 0.0001
NUM_EPOCHS = 100

optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

max_lr = 0.0003

save_path = '/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/mnist_tests_classifier'
NAME = 'classifier__1_9'

scheduler = OneCycleLR(optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

# NUM_EPOCHS * len(train_loader)

#### OneCycleLR Scheduler Parameters:

**max_lr** is the maximum learning rate of OneCycleLR. To be exact, the learning rate will increate from **max_lr** / **div_factor** to **max_lr** in the first **pct_start** * **total_steps** steps, and decrease smoothly to **max_lr** / **final_div_factor** then. div_factor -- Default: 25

**final_div_factor** (float) – Determines the minimum learning rate via min_lr = initial_lr/final_div_factor Default: 1e4

In [ ]:
# max_lr/25

In [ ]:
# len(train_loader.dataset)/train_loader.batch_size

In [ ]:
# # optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9) # Wrapped optimizer
# # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,max_lr=0.9,total_steps=EPOCHS * BATCHES)
# # NUM_EPOCHS = 10
# # learning_rate = 0.0001
# # NUM_BATCHES = math.ceil(len(train_loader.dataset)/train_loader.batch_size)
# # optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)
# # max_lr = 0.0003
# # scheduler = OneCycleLR(optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
# lrs = []
# steps = []
# for epoch in range(NUM_EPOCHS):
#     for batch in range(NUM_BATCHES):
#         scheduler.step()
#         lrs.append(scheduler.get_last_lr()[0])
#         steps.append(epoch * NUM_BATCHES + batch)

# plt.figure()
# plt.legend()
# plt.plot(steps, lrs, label='OneCycle')
# plt.show()

test cnn

In [ ]:
classifier.eval;
img = torch.rand(32, 1, 28, 28)  #image 28 on 28 with 1 chanel like in mnist
# img, label = train_dataset[0]
print(img.size())
print('the size of output model to check the output:', classifier(img).size())


test CNNClassifierWrapper to get feature representations

In [ ]:
from modules.mnist_models import CNNClassifierWrapper

In [ ]:
CNN_wrapper = CNNClassifierWrapper(classifier)

In [ ]:
# classifier.eval;
img = torch.rand(32, 1, 28, 28)  #image 28 on 28 with 1 chanel like in mnist
# img, label = train_dataset[0]
print(img.size())
# print('the size of output model to check the output:', classifier(img).size())
CNN_wrapper.get_nth_layer_result(img).size()

### TRAIN

In [ ]:
train(
    num_epochs = NUM_EPOCHS,
    train_loader = train_loader,
    test_loader = test_loader,
    classifier = classifier,
    optimizer = optimizer,
    criterion = loss_function,
    device = device,
    name = NAME, 
    save_path = save_path,
    plot_process = True,
    info_n = 2,
    scheduler = scheduler,
    save_model_name = 'classifier_model.pt'
    
    # scheduler_D = scheduler_D,
    # scheduler_G = scheduler_G
)

In [ ]:
# classifier = CNN2().to(device=device)
# loss_function = nn.CrossEntropyLoss()

learning_rate = 0.0001
NUM_EPOCHS = 10

optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

max_lr = 0.001

save_path = '/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/mnist_tests_classifier'
NAME = 'classifier__CNN3__1_9_16'

scheduler = OneCycleLR(optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

# NUM_EPOCHS * len(train_loader)

train(
    num_epochs = NUM_EPOCHS,
    train_loader = train_loader,
    test_loader = test_loader,
    classifier = classifier,
    optimizer = optimizer,
    criterion = loss_function,
    device = device,
    name = NAME, 
    save_path = save_path,
    plot_process = True,
    info_n = 2,
    scheduler = scheduler,
    save_model_name = 'classifier_model.pt'
    
    # scheduler_D = scheduler_D,
    # scheduler_G = scheduler_G
)

In [ ]:
# classifier = CNN2().to(device=device)
# loss_function = nn.CrossEntropyLoss()
for i in range(10):
    learning_rate = 0.00003
    NUM_EPOCHS = 10

    optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

    max_lr = 0.0001

    save_path = '/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/mnist_tests_classifier'
    NAME = f'classifier__CNN3__1_9_{i+25}'

    scheduler = OneCycleLR(optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

    # NUM_EPOCHS * len(train_loader)

    train(
        num_epochs = NUM_EPOCHS,
        train_loader = train_loader,
        test_loader = test_loader,
        classifier = classifier,
        optimizer = optimizer,
        criterion = loss_function,
        device = device,
        name = NAME, 
        save_path = save_path,
        plot_process = True,
        info_n = 5,
        scheduler = scheduler,
        save_model_name = 'classifier_model.pt'

        # scheduler_D = scheduler_D,
        # scheduler_G = scheduler_G
    )

### Load model

In [ ]:
from modules.mnist_classifier import eval_model, save_model
from modules.mnist_models import CNN, CNN2

In [ ]:
model = CNN().to(device=device)
# PATH = './mnist_tests_classifier/test/classifier_model.pt'
PATH = './mnist_tests_classifier/classifier__1_5/classifier_model.pt'
model.load_state_dict(torch.load(PATH))
model.eval()

In [ ]:
a, b = eval_model(test_loader, classifier, criterion = loss_function, device = device)
# a, b = eval_model(test_loader, model, criterion = loss_function, device = device)
a, b

In [ ]:

from modules.mnist_classifier import get_preds
from modules.mnist_classifier import calculate_confusion_matrix

In [ ]:
calculate_confusion_matrix(model, test_loader, device)

# y_pred, y_true = get_preds(model, test_loader, device)

In [ ]:
# from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)

In [ ]:
# classifier.eval()
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_true, y_pred))

In [ ]:
# save_model(classifier, save_path = save_path, name = 'test', name2 = 'classifier_model.pt')